# Analyze GiveSendGo data

In [69]:
%load_ext lab_black

In [6]:
import pandas as pd
import json
import numpy as np
import altair as alt
import os
import glob

In [7]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

### Read and concatenate the most recent json files

In [49]:
path_to_json = 'increments/new' 

json_pattern = os.path.join(path_to_json,'*.json')
file_list = glob.glob(json_pattern)

In [50]:
dfs = []
for file in file_list:
    with open(file) as f:
        json_data = (json.loads(f.read()))
        dfs.append(json_data)

In [51]:
all_data = []

for d in dfs:
    df = pd.DataFrame(d["returnData"]['donations'])
    all_data.append(df)

new_df = pd.concat(all_data)

In [52]:
len(new_df)

26750

### Get the previously scraped data

In [55]:
with open("increments/data_increment_536789.json") as f:
    data_backup = json.load(f)

In [56]:
all_data_old = []

for d in data_backup:
    df = pd.DataFrame(d["returnData"]["donations"])
    all_data_old.append(df)

old_df = pd.concat(all_data_old)

### Combine the two dataframes

In [87]:
df = pd.concat([old_df, new_df])

In [94]:
df = df.drop_duplicates().copy()

In [95]:
df.donation_amount = df.donation_amount.astype(float)

In [96]:
df.donation_amount.sum()

7495548.5

In [97]:
df.donation_amount.mean().round(2)

90.27

In [98]:
df.donation_name.count()

83033

In [99]:
df.sort_values("donation_id").head()

,donation_id,campaign_id,donation_amount,donation_comment,donation_conversion_rate,donation_name,donation_anonymous,donation_date,lovecount,likes
0,463549,49000,100.0,We support the Truckers standing up for our freedom.,1.000000,Jeff Brain,0,6 days ago,0,0
0,464203,49000,100.0,God Bless you and Keep you safe! Freedom for All!,1.000000,Edwards,0,6 days ago,0,0
2,464220,49000,25.0,Thank you! God bless you all and keep you safe. Godspeed you on your mission! ❤️😀❤️,1.000000,MaryEllen Stevens,0,6 days ago,0,0
3,464221,49000,10.0,Glory to God.,1.000000,Our turn.,0,6 days ago,0,0
1,464223,49000,20.0,Go Truckers!,1.000000,,0,6 days ago,0,0


### Dates

In [100]:
df.donation_date = (
    df.donation_date.str.replace("6 days ago", "2022-02-02", regex=False)
    .str.replace("5 days ago", "2022-02-03", regex=False)
    .str.replace("4 days ago", "2022-02-04", regex=False)
    .str.replace("3 days ago", "2022-02-05", regex=False)
    .str.replace("2 days ago", "2022-02-06", regex=False)
    .str.replace("1 days ago", "2022-02-07", regex=False)
)

In [102]:
df['donation_date'] = np.where(~df['donation_date'].str.contains("2022", na=False), "2022-08-08 or later", df['donation_date'])

In [116]:
dates = (
    df.groupby(["donation_date"])
    .agg({"donation_id": "count", "donation_amount": sum})
    .reset_index()
)

In [117]:
dates.rename(columns={"donation_id": "count", "donation_amount": "sum"}, inplace=True)

In [118]:
dates

,donation_date,count,sum
0,2022-02-02,61,3348.0
1,2022-02-03,112,6651.5
2,2022-02-04,115,5424.0
3,2022-02-05,2965,311979.0
4,2022-02-06,17561,1923301.0
5,2022-02-07,32499,2793103.0
6,2022-08-08 or later,29720,2451742.0


In [122]:
alt.Chart(dates).mark_bar(width=20).encode(
    x=alt.X(
        "donation_date:T",
        axis=alt.Axis(format="%b. %d", tickCount=6),
        title="Donation date",
    ),
    y=alt.Y("sum", axis=alt.Axis(tickCount=6), title="Dollars raised"),
)

alt.Chart(...)

---

In [123]:
df["donation_name"] = df["donation_name"].str.strip().str.lower()

In [124]:
df.loc[df["donation_anonymous"] > 1, "donation_name"] = "anonymous"

### Group to count and sum donations by donor name

In [125]:
df.groupby(["donation_name"]).agg(
    {"donation_amount": [np.size, np.sum]}
).reset_index().sort_values(("donation_amount", "sum"), ascending=False).head(20)

donation_name donation_amount           
                                                     size        sum
0                                                   42011  3479244.5
24204         processed but not recorded.               1   215000.0
8650      easy kleen pressure systems ltd               1    75000.0
1586                            anonymous             499    42931.0
10057                             freedom             324    25906.0
12345             holden and carey rhodes               1    25000.0
32249           www the range langley com               1    18000.0
993    american cryptocurrency compatriot               1    17760.0
10232                       freedom lover              82    14856.0
10095                      freedom convoy              92     9778.0
12399                           honk honk              55     9535.0
24001        pierre elite trudo blackface               1     9000.0
16092                      justin trudeau             104     7625.0
9841                       francis lepine               1     7000.0
15004                                john              57     5749.0
1132                          and we know               1     5500.0
3647                        broalbexpress               1     5500.0
16459                    katherine newman               1     5000.0
3364                                breez               1     5000.0
16769                          kelly gave               1     5000.0

In [126]:
df.groupby(["donation_name"]).agg(
    {"donation_amount": [np.size, np.sum]}
).reset_index().sort_values(("donation_amount", "size"), ascending=False).head(20)

donation_name donation_amount           
                                  size        sum
0                                42011  3479244.5
1586         anonymous             499    42931.0
10057          freedom             324    25906.0
16092   justin trudeau             104     7625.0
10095   freedom convoy              92     9778.0
10232    freedom lover              82    14856.0
28041            steve              73     4498.0
21155             mike              60     4233.0
6874             david              59     3899.0
15004             john              57     5749.0
12399        honk honk              55     9535.0
18186  lets go brandon              52     3041.0
19773             mark              49     2527.0
4889             chris              47     3266.0
10115  freedom fighter              39     3370.0
16999            kevin              39     2275.0
20779          michael              38     4853.0
1570              anon              38     1805.0
20292             matt              37     1905.0
18491             lisa              37     3916.0

In [127]:
# 100 biggest donations
# 100 first donations

In [128]:
df.sort_values("donation_amount", ascending=False).head(100).to_csv(
    "data/processed/100_largest_donations.csv", index=False
)

In [129]:
df.sort_values("donation_date", ascending=True).head(100).to_csv(
    "data/processed/100_earliest_donations.csv", index=False
)

In [130]:
df.sort_values("donation_date", ascending=True).to_csv(
    "data/processed/all_donations.csv", index=False
)